# Hierarchical Indexing
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRAQ6O6OHqzjq435DHfq5WT6Nx979RdKLv242B5xSKuKOtpytntDw&s)

In [1]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series with Pandas MultiIndex

Let's start by considering how we might represent two-dimensional data within a one-dimensional ``Series``.
For concreteness, we will consider a series of data where each point has a character and numerical key.

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]

In [3]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [4]:
pop = pd.Series(populations, index=index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Slicing

In [5]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

The result is a singly indexed array with just the keys we're interested in.

### MultiIndex as extra dimension
![](https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_unstack.png)

In [6]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


![](https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_stack.png)

In [7]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## Ufuncs work with hierarchical indices

In [8]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [9]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [10]:
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## Methods of MultiIndex Creation
### Pass a list of two or more index arrays to the constructor

In [11]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.755604  0.453315
  2  0.398117  0.577115
b 1  0.107123  0.425775
  2  0.471748  0.408881

#### Pass a dictionary with appropriate tuples as keys

In [12]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Explicit MultiIndex constructors with ``pd.MultiIndex``

In [13]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [14]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [15]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Similarly, you can construct the ``MultiIndex`` directly using its internal encoding by passing ``levels`` (a list of lists containing available index values for each level) and ``codes`` (a list of lists that reference these codes):

In [16]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

#### Any of these objects can be:
- passed as the ``index`` argument when creating a ``Series`` or ``Dataframe``, or 
- passed to the ``reindex`` method of an existing ``Series`` or ``DataFrame``.

### MultiIndex level names

In [17]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for columns

In [18]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
# This is fundamentally four-dimensional data, 
# where the dimensions are the subject, the measurement type, the year, and the visit number.
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      48.0  36.4  39.0  36.7  51.0  36.2
     2      39.0  37.2  45.0  35.9  40.0  36.9
2014 1      37.0  38.2  40.0  36.8  58.0  37.7
     2      35.0  36.4  40.0  36.7  32.0  37.7

#### Indexing

In [19]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      39.0  36.7
     2      45.0  35.9
2014 1      40.0  36.8
     2      40.0  36.7

For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!

## Indexing and Slicing a MultiIndex

### Multi-indexed Series

In [20]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

We can access single elements by indexing with multiple terms:

In [21]:
pop['California', 2000]

33871648

#### *Partial indexing*: indexing just one of the levels in the index
The result is another ``Series``, with the lower-level indices maintained:

In [22]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

#### Partial slicing is available as well, as long as the ``MultiIndex`` is sorted
See **Rearranging Multi-Indices** below

In [23]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:

In [24]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

#### Selection based on Boolean masks:

In [25]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

#### Selection based on fancy indexing:

In [26]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multi-indexed DataFrames

In [27]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      48.0  36.4  39.0  36.7  51.0  36.2
     2      39.0  37.2  45.0  35.9  40.0  36.9
2014 1      37.0  38.2  40.0  36.8  58.0  37.7
     2      35.0  36.4  40.0  36.7  32.0  37.7

In [28]:
health_data['Guido', 'HR']

year  visit
2013  1        39.0
      2        45.0
2014  1        40.0
      2        40.0
Name: (Guido, HR), dtype: float64

### ``loc`` & ``iloc`` also work

In [29]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      48.0  36.4
     2      39.0  37.2

In [30]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        48.0
      2        39.0
2014  1        37.0
      2        35.0
Name: (Bob, HR), dtype: float64

# Rearranging Multi-Indices

## Sorted and unsorted indices
*Many of the ``MultiIndex`` slicing operations will fail if the index is not sorted.*  
Let's take a look at this here.  
We'll start by creating some simple multiply indexed data where the indices are *not lexographically sorted*:

In [31]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.235458
      2      0.675250
c     1      0.736999
      2      0.059815
b     1      0.914058
      2      0.012164
dtype: float64

If we try to take a partial slice of this index, it will result in an error:

In [32]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


This is the result of the MultiIndex not being sorted.  
For various reasons, partial slices and other similar operations require the levels in the ``MultiIndex`` to be in sorted (i.e., lexographical) order.

## ``sort_index()`` & ``sortlevel()`` methods of the ``DataFrame``

In [33]:
data = data.sort_index()
data

char  int
a     1      0.235458
      2      0.675250
b     1      0.914058
      2      0.012164
c     1      0.736999
      2      0.059815
dtype: float64

With the index sorted in this way, partial slicing will work as expected:

In [34]:
data['a':'b']

char  int
a     1      0.235458
      2      0.675250
b     1      0.914058
      2      0.012164
dtype: float64

## Stacking and unstacking indices
![](https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_unstack_0.png)

In [35]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [36]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


![](https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_stack.png)

In [37]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## Index resetting: ``reset_index``

In [38]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [39]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


## Index setting: ``set_index``

In [40]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## Data Aggregations on Multi-Indices

In [41]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      48.0  36.4  39.0  36.7  51.0  36.2
     2      39.0  37.2  45.0  35.9  40.0  36.9
2014 1      37.0  38.2  40.0  36.8  58.0  37.7
     2      35.0  36.4  40.0  36.7  32.0  37.7

In [42]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
year                                         
2013     43.5  36.8  42.0  36.30  45.5  36.55
2014     36.0  37.3  40.0  36.75  45.0  37.70

In [43]:
data_mean.mean(axis=1, level='subject')

subject,Bob,Guido,Sue
year,,,
2013,40.15,39.150,41.025
2014,36.65,38.375,41.350


In [44]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,43.666667,36.55
2014,40.333333,37.25
